# <center>第五章 蒙特卡洛法</center>
&emsp;&emsp;动态规划法属于基于模型的MDP问题求解方法。在环境模型已知的情况下，动态规划法无需环境采样，只要通过迭代计算，就可以得到问题的最优策略。然而在实际任务中，完整的环境模型通常是难以获取的。无模型的强化学习，状态转移概率是未知的，也就是说无模型强化学习无法利用动态规划方法来求解值函数。于是可以尝试通过值函数的原始定义来求解无模型强化学习问题：<br>
$$\begin{aligned}
v_{\pi}&=\mathbb{E}_{\pi}(G_t|S_t=s) \end{aligned}$$
$$\begin{aligned}
q_{\pi}(s,a)&=\mathbb{E}_{\pi}(G_t|S_t=s,A_t=a) \end{aligned}$$
&emsp;&emsp;本章将主要介绍如何利用蒙特卡洛法（Monte Carlo，MC）来求解无模型强化学习问题，以GPI来保证其最优性。这里首先考虑预测问题中的策略改进问题，然后再考虑控制问题中的策略改进问题。<br>

## 5.1 蒙特卡洛法的基本概念
&emsp;&emsp;在实际问题中，通常不易获得完整的环境知识。例如在21点游戏中，只根据玩家已知的牌面，无法直接获得状态转移概率。相对而言，采样数据则更容易获得。比如可以多次进行游戏对弈，然后估计专家的叫停牌套路，计算出最优的游戏策略。这种基于统计学的思想，通过大量采样获取数据来进行学习的方法称为经验方法。MC正是基于经验方法，在环境模型未知的情况下，采用时间步有限的、完整的情节，根据经验进行学习，并通过平均采样回报来解决强化学习问题。<br>
### 5.1.1 MC的核心要素
&emsp;&emsp;（1）经验（experience）：经验是从环境交互中获得的$(s,a,r)$序列，它是情节的集合，也就是样本集。经验既可以是真实经验（real experience），也可以是模拟经验（simulater experience）。其中模拟经验是通过模拟模型（simulated model）得到的，这里的模拟模型只需生成状态转移的一些样本，无需像DP那样需要环境中所有可能的状态转移概率。<br>
&emsp;&emsp;（2）情节（episode）：一段经验可以分为多个情节，每一情节都是一个完整的$(s,a,r)$序列，即必有终止状态，形如$s_0,a_0,r_1,...,s_{T-1},a_{T-1},r_T,s_T$。经常与情节混淆的概念是轨迹（trajectory），轨迹可以不存在终止状态，形如 $s_0,a_0,r_1,s_1,a_1,r_2,...$。所以通常情况下，可以认为：<br>
$$\begin{aligned}
序列\subseteq情节\subseteq经验（轨迹） \end{aligned}$$
&emsp;&emsp;（3）完整回报（full return）与目标值：因为只有到达终止状态才能计算回报，所以将情节的回报$G_t$称为完整回报。此外， $G_t$也称为MC的目标值。<br>
### 5.1.2 MC的特点
&emsp;&emsp;（1）无需知道状态转移概率$p$，直接从环境中进行采样来处理无模型任务。<br>
&emsp;&emsp;（2）利用情节进行学习，并采用情节到情节（episode-by-episode）的离线学习（off-line）方式来求解最优策略$\pi_\ast$。与之相对的，DP和后续介绍的时序差分算法则采用步到步（step-by-step）的在线学习（on-line）方式来求解最优策略。<br>
&emsp;&emsp;&emsp;$\blacktriangleright$&emsp;离线学习：先完整地采集数据，然后以离线方式优化学习目标。<br>
&emsp;&emsp;&emsp;$\blacktriangleright$&emsp;在线学习：边采集数据边优化学习目标。<br>
&emsp;&emsp;（3）MC是一个非平稳问题，其表现在：某个状态采取动作之后的回报，取决于在同一个情节内后续状态所采取的动作，而这些动作通常是不确定的。如果说DP是在MDP模型中计算值函数，那么MC就是学习值函数。<br>
&emsp;&emsp;（4）在MC中，对每个状态的值函数估计都是独立的。也就是说，对状态的值函数估计不依赖于其他任何状态，这也说明了MC不是自举过程。<br>
&emsp;&emsp;（5）MC在估计每个状态的值函数时，其计算成本与状态总数无关，因为它只需要计算指定状态的回报，无需考虑所有的状态。<br>
&emsp;&emsp;实际上，MC泛指任何包含大量随机成分的估计方法，通常利用采样数据来估算某一事件的发生概率。在数学领域中，它的应用可以用例5.1来说明。<br>
&emsp;&emsp;<b>例5.1</b> 在边长为1米的正方形$S_1$内构建一个扇形$S_2$，如图5.1所示。利用扇形面积计算公式，可以计算出$S_2=\frac{1}{4}\pi r^{2}\approx\frac{1}{4}\times3.14\times 1^{2}=0.785m^{2}$。现在利用MC方法计算$S_2$的面积。均匀地向$S_1$内撒$n$个黄豆，经统计得知：有$m$个黄豆在$S_2$内部，那么有$\frac{S_2}{S_1}\approx\frac{m}{n}$，即$S_2\approx\frac{m}{n}S_1$，且$n$越大，计算得到的面积越精确。<br>
<center><img src='./image/图5.1 用MC方法计算扇形面积.png' width='300' height='300'></center>
<center>图5.1&ensp;用MC方法计算扇形面积</center>
&emsp;&emsp;在程序中分别设置$n=100、10000$和$1000000$共3组数据，统计结果如表5.1所示。<br>

<table align="center" border="3" bgcolor="#DC143C" cellspacing="1" cellpadding="5px" width="100%">
    <caption>
        <font size=2><center>表5.1 MC方法计算扇形面积结果统计表</center></font>
    </caption>
    <tr>
        <th width="120px"><p style="text-align:center">&emsp;&emsp;</p></th>
        <th><p style="text-align:center">$S_{1}$</p></td>
        <th><p style="text-align:center">$S_{2}$</p></td>
        <th><p style="text-align:center">$m$</p></td>
        <th><p style="text-align:center">$m/n$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$n=100$</p></td>
        <td><p style="text-align:center">1.0</p></td>
        <td><p style="text-align:center">0.785</p></td>
        <td><p style="text-align:center">79.0</p></td>
        <td><p style="text-align:center">0.79</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$n=10000$</p></td>
        <td><p style="text-align:center">1.0</p></td>
        <td><p style="text-align:center">0.785</p></td>
        <td><p style="text-align:center">7839.0</p></td>
        <td><p style="text-align:center">0.7839</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$n=1000000$</p></td>
        <td><p style="text-align:center">1.0</p></td>
        <td><p style="text-align:center">0.785</p></td>
        <td><p style="text-align:center">786018.0</p></td>
        <td><p style="text-align:center">0.786018</p></td>
    </tr>
</table>

&emsp;&emsp;由实验数据可知，当$n$的值越大时，得到的扇形面积越精确，即接近0.785。<br>
&emsp;&emsp;由此获得的MC采样公式为：<br>
$$
\mathbb{E}_{x-p}[f(x)]=\sum_x p(x)f(x)\approx\frac{1}{N}\sum\limits_{x_i\sim p,i=1}^{N} f(x_i) \tag{5.1}$$

## 5.2 蒙特卡洛预测
&emsp;&emsp;根据状态值函数的初始定义，MC预测算法以情节中初始状态$s$的回报期望作为其值函数$v_\pi(s)$的估计值，对策略$\pi$进行评估。在求解状态$s$的值函数时，先利用策略$\pi$产生$n$个情节$s_0,a_0,r_1,...,s_{T-1},a_{T-1},r_T,s_T$，然后计算每个情节中状态$s$的折扣回报：<br>
$$G_{t}^{(i)}=r_t+\gamma r_{t+1}+...+\gamma^{T-t}r_T$$
这里，$G_{t}^{(i)}$表示在第$i$个情节中，从$t$时刻到终点时刻$T$的回报。该回报是基于某一策略下的状态值函数的无偏估计（由于$G_t$是真实获得的，所以属于无偏估计，但是存在高方差）。<br>
&emsp;&emsp;在MC中，每个回报都是对$v_\pi(s)$独立同分布的估计，通过对这些折扣回报求期望（均值）来评估策略$\pi$：<br>
$$v_\pi(s)=\mathbb{E}_{\pi}(G_t\mid s\in S)=average(G_{t}^{(1)}+G_{t}^{(2)}+...+G_{t}^{(i)}+...+G_{t}^{(n)}\mid s\in S)$$
&emsp;&emsp;在一组采样（一个情节）中状态$s$可能多次出现，以更新图的方式表示，如图5.2所示。对同一情节中重复出现的状态$s$，有如下两种处理方法：<br>
&emsp;&emsp;（1）首次访问（first-visit）：在对状态$s$的回报$v_\pi(s)$进行估计时，只对每个情节中第1次访问到状态$s$的回报值作以统计：<br>
$$ V(s)=\frac{G_{t}^{(1)(1)}(s)+G_{t}^{(2)(1)}(s)+...+G_{t}^{(i)(1)}(s)}{i} \tag{5.2}$$
&emsp;&emsp;(2)每次访问（every-visit）：在对状态$s$的回报$v_\pi(s)$进行估计时，对所有访问到状态$s$的回报值都作以统计：<br>
$$ V(s)=\frac{G_{t}^{(1)(1)}(s)+G_{t}^{(1)(2)}(s)+...+G_{t}^{(2)(1)}(s)+...+G_{t}^{(i)(1)}(s)+...+G_{t}^{(i)(j)}(s)}{N(s)} \tag{5.3}$$
其中，$i$表示第$i$个情节，$j$表示第$j$次访问到状态$s$； $N(s)$表示状态$s$被访问过的总次数。根据大数定理，当MC采集的样本足够多时，计算出来的状态值函数估计值$V_\pi(s)$就会逼近真实状态值函数$v_\pi(s)$。
<center><img src='./image/图5.2 在情节中状态s的出现情况（MC更新图）.png' width='500' height='500'></center>
<center>图5.2&ensp;在情节中状态s的出现情况（MC更新图）</center>
&emsp;&emsp;从图5.2中可以看出，MC更新图只显示在当前情节中，采样得到的状态转移，且包含了到该情节结束为止的所有状态转移；而DP更新图显示在当前状态下所有可能的状态转移，且仅包含单步转移。<br>

&emsp;&emsp;基于首次访问的MC预测算法，如算法5.1所示。<br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法5.1</b> 基于首次访问的MC预测算法</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输入：</b></font>
<font size=3.5>待评估的策略$\pi(a\mid s)$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
&emsp;<font size=3.5>1.&emsp;对任意$s\in\mathcal{S}$，初始化$V(s)\in\mathbb{R}$，$V(s^T)=0$；状态$s$被统计的次数$count(s)=0$</font><br>
<hr>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;<b>repeat</b> 对每个情节$k=0，1，2，\cdots$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>3.&emsp;&emsp;&emsp;根据策略$\pi(a\mid s)$，生成一个情节序列：$S_0,A_0,R_1,...,S_{T-1},A_{T-1},R_T,S_T$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;&emsp;&emsp;$G\gets0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>5.&emsp;&emsp;&emsp;<b>for</b> 本情节中的每一步$t=T-1$ <b>downto</b>&emsp;0&emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;&emsp;&emsp;$G\gets\gamma G+R_{t+1}$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;&emsp;&emsp;<b>if</b> $S_t\notin\{S_0,S_1,...,S_{T-1}\}$ <b>then</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$V(S_t)\gets V(S_t)+\frac{1}{count(S_t)}(G-V(S_t))$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$count(S_t)\gets count(S_t)+1$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>10.&emsp;&emsp;&emsp;&emsp;&emsp;<b>end if</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>11.&emsp;&emsp;&emsp;<b>end for</b></font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 出：</b></font>
<font size=3.5>$v_\pi=V$</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>

&emsp;&emsp;<b>例5.2</b> 以例3.1扫地机器人为例。给出机器人经过图5.3的每条轨迹后，相对应的状态值。<br>
<center><img src='./image/图5.3 经过状态16的其中5个情节.png' width='700' height='700'></center>
<center>图5.3&ensp;经过状态16的其中5个情节</center>
&emsp;&emsp;如图5.3所示，选取了5个经过状态16的情节，5个情节依次设置为情节1、情节2、情节3、情节4和情节5。<br>
&emsp;&emsp;情节1：$16\to15\rightarrow10\rightarrow5\rightarrow0$<br>
&emsp;&emsp;$G_{16}^{(1)(1)}=0+0.8\times(0+0.8\times(0+0.8\times(1+0.8\times0)))=0.8^3\times1=0.512$<br>
&emsp;&emsp;情节2：$16\rightarrow17\rightarrow17\rightarrow18\rightarrow19$<br>
&emsp;&emsp;$G_{16}^{(2)(1)}=0+0.8\times(-10+0.8\times(0+0.8\times(3+0.8\times0)))=0.8\times(-10)+0.8^3\times3=-8.464$<br>
&emsp;&emsp;情节3：$16\rightarrow11\rightarrow6\rightarrow7\rightarrow8\rightarrow9\rightarrow14\rightarrow19$<br>
&emsp;&emsp;$G_{16}^{(3)(1)}=0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(3+0.8\times0))))))=0.8^6\times3=0.786$<br>
&emsp;&emsp;也可以直接利用关于回报的定义计算：<br>
&emsp;&emsp;$G_{16}^{(3)(1)}=0+0.8\times0+0.8^2\times0+0.8^3\times0+0.8^4\times0+0.8^5\times0+0.8^6\times3=0.8^6\times3=0.786$<br>
&emsp;&emsp;情节4：$16\rightarrow11\rightarrow6\rightarrow7\rightarrow8\rightarrow13\rightarrow18\rightarrow19$<br>
&emsp;&emsp;$G_{16}^{(4)(1)}=0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(3+0.8\times0))))))=0.8^6\times3=0.786$<br>
&emsp;&emsp;情节5：首次访问状态16：$16\rightarrow11\rightarrow6\rightarrow7\rightarrow8\rightarrow13\rightarrow18\rightarrow17\rightarrow16\rightarrow15\rightarrow10\rightarrow5\rightarrow1$<br>
$G_{16}^{(5)(1)}=0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(0+0.8\times(1+0.8\times0)))))))))))=0.8^{11}\times1=0.086$<br>
&emsp;&emsp;第2次访问状态16：$16\rightarrow15\rightarrow10\rightarrow5\rightarrow1$<br>
$G_{16}^{(5)(2)}=0+0.8\times(0+0.8\times(0+0.8\times(1+0.8\times0)))=0.8^{3}\times1=0.512$<br>
&emsp;&emsp;在情节$5$中状态$16$被访问了两次。利用首次访问的MC预测方法时，计算累计回报值只使用该情节中第一次访问状态$16$的回报，即$G_{16}^{(5)(1)}$。所以使用这$5$条情节，利用首次访问方式计算状态$16$的估计值为：$V(S_{16})=(G_{16}^{(1)(1)}+G_{16}^{(2)(1)}+G_{16}^{(3)(1)}+G_{16}^{(4)(1)}+G_{16}^{(5)(1)})/5=-1.166$；而利用每次访问方式计算状态$16$的估计值为：$V(S_{16})=(G_{16}^{(1)(1)}+G_{16}^{(2)(1)}+G_{16}^{(3)(1)}+G_{16}^{(4)(1)}+G_{16}^{(5)(1)}+G_{16}^{(5)(2)})/6=-0.886$。<br>

## 5.3 蒙特卡洛评估
&emsp;&emsp;由最优策略的两种求解方式可知，利用动作值函数是一种更适合于无模型求解最优策略的方法。于是将估计状态值函数的MC预测问题转化为估计动作值函数的MC评估问题，也就是说，对状态-动作对$(s,a)$进行访问而不是对状态$s$进行访问。根据策略$\pi$进行采样，记录情节中$(s,a)$的回报$G_t$，并对$(s,a)$的回报求期望，得到策略$\pi$下的动作值函数$q_\pi(s,a)$的估计值。同样地，MC评估方法对每一组状态-动作对$(s,a)$的评估方法也分为首次访问和每次访问两种。<br>
&emsp;&emsp;为了保证算法中值函数和策略的收敛性，DP算法会对所有状态进行逐个扫描。在MC评估方法中，根据动作值函数计算的性质，必须保证每组状态-动作对$(s,a)$都能被访问到，即得到所有$(s,a)$的回报值，才能保证样本的完备性。针对该问题，我们设定探索始点（exploring starts）：每一组$(s,a)$都有非0的概率作为情节的起始点$(s_0,a_0)$。<br>
&emsp;&emsp;实际上，探索始点在实际应用中是难以达成的，需要配合无限采样才能保证样本的完整性。通常的做法是采用那些在每个状态下所有动作都有非0概率被选中的随机策略。这里我们先从简单的满足探索始点的MC控制算法开始讨论，然后引出基于同策略和异策略方法的MC控制算法。<br>
## 5.4 蒙特卡洛控制
&emsp;&emsp;预测和控制的思想是相同的，它们都是基于带奖励过程的马尔可夫链来对目标进行更新，其区别在于：<br>
&emsp;&emsp;（1）MC预测：求解在给定策略$\pi$下，状态$s$的状态值函数$v_\pi(s)$。<br>
&emsp;&emsp;（2）MC控制：基于GPI，包含策略评估和策略改进两部分。这里的策略评估是求解在给定策略$\pi$下，状态-动作对$(s,a)$的动作值函数$q_\pi(s,a)$。其策略迭代过程如下所示：<br>
$$\pi_0\stackrel{E}{\longrightarrow}q_{\pi_0}\stackrel{I}{\longrightarrow}\pi_1\stackrel{E}{\longrightarrow}q_{\pi_1}\stackrel{I}{\longrightarrow}...\stackrel{I}{\longrightarrow}\pi_\ast\stackrel{E}{\longrightarrow}q_{\ast}$$
### 5.4.1 探索与利用
&emsp;&emsp;由于强化学习的目的是得到最优策略，使得回报最大化，这就面临到强化学习的一大矛盾：探索与利用的平衡。一方面，Agent秉持利用（exploitation）机制，为了得到最大回报，需要始终采用最优动作，即根据当前的值函数选择最优动作，最大限度地提升回报。另一方面，Agent需要探索（exploration）机制，摒弃基于值函数的贪心策略，找到更多可能的动作来获得更好的策略，探索更多的可能性。<br>
&emsp;&emsp;通常我们采用具有探索性的策略来解决以上矛盾，常用的两种方法为同策略和异策略。使用这两种方法一方面保证足够的探索性，另一方面充分地利用最优策略。在介绍它们之前，先来了解行为策略和目标策略：<br>
&emsp;&emsp;（1）行为策略（behavior policy）：用于产生采样数据的策略，具备探索性，能够覆盖所有情况，通常采用$\varepsilon-$柔性策略（$\varepsilon-$soft policy）。<br>
&emsp;&emsp;（2）目标策略（target policy）：强化学习任务待求解的策略，也就是待评估和改进的策略，一般不具备探索性，通常采用确定性贪心策略（deterministic greedy policy）。<br>
&emsp;&emsp;同策略和异策略的特点分别阐述如下：<br>
&emsp;&emsp;（1）同策略（on-policy）：行为策略和目标策略相同。通过$\varepsilon-$贪心策略平衡探索和利用，在保证初始状态-动作对$(S_0,A_0)$不变的前提下，确保每一组$(s,a)$都有可能被遍历到。常用算法为Sarsa和Sarsa(𝜆)算法。<br>
&emsp;&emsp;（2）异策略（off-policy）：行为策略和目标策略不同。将探索与利用分开，在行为策略中贯彻探索原则：采样数据，得到状态-动作序列。在目标策略中贯彻利用原则：更新值函数并改进目标策略，以得到最优目标策略。常用算法为Q-learning和DQN算法。<br>
&emsp;&emsp;同策略方法较为简单，通常会被优先考虑，但异策略方法更为通用，效果也更好，可以通过传统的非学习型控制器或人类专家生成的数据来学习。异策略方法的优缺点分别阐述如下：<br>
&emsp;&emsp;异策略优点：<br>
&emsp;&emsp;$\blacktriangleright$&emsp;效果好，更具有一般性，可以从示例样本或其他Agent给出的数据样本中进行学习。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;可以重复利用旧策略。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;可以在使用一个探索性策略的基础上，学习一个确定性策略。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;可以用一个行为策略进行采样，多个目标策略同时进行学习。<br>
&emsp;&emsp;异策略缺点：<br>
&emsp;&emsp;由于数据来源于一个不同的策略，存在方差较大、收敛速度慢的缺陷。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;偏差（bias）：<br>
&emsp;&emsp;在机器学习中，偏差和欠拟合相关，预测值与样本之间的偏差越大，说明精度越低。在强化学习中，偏差指通过采样数据获得的估计平均值与实际平均值的偏离程度。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;方差（variance）：<br>
&emsp;&emsp;在机器学习中，方差和过拟合相关，样本值之间的方差越大，说明样本的置信度越差，模型适用性越差。在强化学习中，方差指单次采样结果相对于均值的波动大小。<br>
### 5.4.2 基于探索始点的蒙特卡洛控制
&emsp;&emsp;当采样过程满足探索始点时，对任意策略$\pi_k$，MC算法都可以准确地计算出指定$(s,a)$的动作值函数$q_{\pi_k}(s,a)$。一旦得到了动作值函数，可以直接利用基于动作值函数的贪心策略来对策略进行更新。此时对于所有$s\in \mathcal{S}$，任意策略$\pi_k$以及更新后的$\pi_{k+1}$都将满足策略改进原理：<br>
$$\begin{aligned}
q_{\pi_k}(s,\pi_{k+1}(s)) &=q_{\pi_k}(s,arg\max_aq_{\pi_k}(s,a)) &根据\pi_{k+1}=arg\max_aq_{\pi_k}(s,a)\\
&= \max_aq_{\pi_k}(s,a)       & 将最优策略提到公式外面使用\\
&\geqslant q_{\pi_k}(s,\pi_k(s)) &上一个策略的动作值函数\\
&\geqslant v_{\pi_k}(s)
\end{aligned}  \tag{5.4}$$
&emsp;&emsp;式（5.4）表明，采用基于动作值函数的贪心策略，改进后的策略$\pi_{k+1}$一定优于或等价于$\pi_k$。这一过程保证了MC控制算法能够收敛到最优动作值函数和最优策略。<br>
&emsp;&emsp;对于5.3节介绍的无穷采样假设，由于在实际环境中无法实现这一条件，我们使用基于探索始点的蒙特卡洛（Monte Carlo based on Exploring States，MCES）控制算法来进行规避。MCES控制算法通过情节到情节的方式对策略进行评估和更新，每一情节结束后，使用观测到的回报进行策略评估，然后在该情节访问到的每一个状态上进行策略改进。<br>
&emsp;&emsp;（1）策略评估：根据当前策略$\pi$进行采样，得到一条完整情节，估计每一组$(s,a)$的动作值函数；<br>
&emsp;&emsp;（2）策略改进：基于动作值函数$q_\pi(s,a)$，直接利用贪心策略对策略进行改进。<br>

&emsp;&emsp;算法5.2给出了基于探索始点的蒙特卡洛控制——MCES算法。<br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法5.2</b>  MCES控制算法</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输入：</b></font>
<font size=3.5>待评估的策略$\pi(s)$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
<font size=3.5>1.&emsp;对所有$s\in\mathcal{S}^+，a\in\mathcal{A}(s)$，初始化$Q（s,a）\in\mathbb{R}$，$Q(s^T,a)=0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;对所有$s\in\mathcal{S}^+，a\in\mathcal{A}(s)$，状态-动作对$(s,a)$被统计的次数$count(s,a)=0$</font><br>
<hr>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>3.&emsp;<b>repeat</b> 对每个情节$k=0，1，2，\cdots$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;&emsp;&emsp;以非0概率随机选取初始状态-动作对$(S_0,A_0)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>5.&emsp;&emsp;&emsp;根据策略$\pi(s)$，从初始状态-动作对$(S_0,A_0)$开始，生成一个情节序列：$S_0,A_0,R_1,...,S_{T-1},A_{T-1},R_T,S_T$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;$G\gets0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;<b>for</b> 本情节中的每一步$t=T-1$ <b>downto</b>&emsp;0&emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;&emsp;&emsp;$G\gets\gamma G+R_{t+1}$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;&emsp;&emsp;&emsp;<b>if</b> $S_t,A_t\notin\{S_O,A_0,S_1,A_1,...,S_{T-1},A_{T-1}\}$ <b>then</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>10.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$Q(S_t,A_t)\gets Q(S_t,A_t)+\frac{1}{count(S_t,A_t)}(G-Q(S_t,A_t))$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>11.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$count(S_t,A_t)\gets count(S_t,A_t)+1$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>12.&emsp;&emsp;&emsp;&emsp;&emsp;<b>end if</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>13.&emsp;&emsp;&emsp;&emsp;&emsp;$\pi(S_t)\gets arg\max\limits_a Q(S_t,a)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>14.&emsp;&emsp;&emsp;<b>end for</b></font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输 出：</b></font>
&emsp;<font size=3.5>$\pi_\ast=\pi$</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>

&emsp;&emsp;在算法5.2中，$Q(s,a)$的初始值一般设置为0。若使用非0初始化，则可以通过$Q[s][a]=random()/10$来控制。$Q(s,a)$表示动作值函数估计值，所以使用大写$Q$表示。在程序中，常以$defalutdict( )$声明未定型字典$Q(s,a)$，其参数可表示为：$\{s:(action1\_value,action2\_value),....\}$形式，并通过$Q[s][a]$来调用$(s,a)$的动作值函数。<br>
&emsp;&emsp;虽然可以通过探索性始点来弥补无法访问到所有状态-动作对的缺陷，但这一方法并不合理，唯一普遍的解决方法就是保证Agent能够持续不断地选择所有可能的动作，这也称为无探索始点方法，该方法分为同策略方法与异策略方法两种。<br>

### 5.4.3 同策略蒙特卡洛控制
&emsp;&emsp;在同策略MC控制算法中，策略通常是软性的（soft），即对于所有的$s\in\mathcal{S}$和$a\in\mathcal{A}(s)$，均有$\pi(a\mid s)>0$。但策略都必须逐渐变得贪心，以逐渐逼近一个确定性策略。同策略方法使用$\varepsilon-$贪心策略（$\varepsilon-$greedy policy），其公式为：<br>
$$\pi(a\mid s)\gets\begin{cases}
1-\varepsilon+\frac{\varepsilon}{\mid\mathcal{A}(s)\mid} & 当a=A^*时，以概率1-\varepsilon+\frac{\varepsilon}{\mid\mathcal{A}(s)\mid}选择具有最大价值的动作\\
\frac{\varepsilon}{\mid\mathcal{A}(s)\mid} & 当a\neq A^*时，以概率\frac{\varepsilon}{\mid\mathcal{A}(s)\mid}随机选择动作
\end{cases} \tag{5.5}$$
其中，$\mathcal{A}(s)$表示状态$s$的动作空间，$\mid\mathcal{A}(s)\mid$表示状态$s$可采取的动作总数；$A^*$表示最优动作。<br>
&emsp;&emsp;GPI并没有要求必须使用贪心策略，只要求采用的优化方法逐渐逼近贪心策略即可。根据策略改进定理，对于一个$\varepsilon-$软性策略$\pi$，任何根据$q_\pi$生成的$\varepsilon-$贪心策略都是对其所做的改进。下面对$\varepsilon-$软性策略改进定理进行证明。<br>
&emsp;&emsp;假设$\pi^{'}$为$\varepsilon-greedy$策略，对任意状态$s\in\mathcal{S}$有：<br>
$$\begin{aligned}
q_{\pi}(s,\pi^{'}(s)) &=\sum_a \pi^{'}(s\mid a)q_\pi(s,a)\\
&=\frac{\varepsilon}{\mid\mathcal{A}(s)\mid}\sum_aq_\pi(s,a)+(1-\varepsilon)\max_aq_\pi(s,a)\\
&\geqslant \frac{\varepsilon}{\mid\mathcal{A}(s)\mid}\sum_aq_\pi(s,a)+(1-\varepsilon)\frac{\pi(a\mid s)-\frac{\varepsilon}{\mid\mathcal{A}(s)\mid}}{1-\varepsilon}q_\pi(s,a) \\
&= \frac{\varepsilon}{\mid\mathcal{A}(s)\mid}\sum_aq_\pi(s,a)-\frac{\varepsilon}{\mid\mathcal{A}(s)\mid}\sum_aq_\pi(s,a)+\sum_a\pi(a\mid s)q_\pi(s,a) \\
&\geqslant v_\pi(s)
\end{aligned}  $$
&emsp;&emsp;所以，根据策略改进定理，$\pi^{'}\ge\pi$。<br>

&emsp;&emsp;基于同策略首次访问$\varepsilon-greedy$策略的MC算法，如算法5.3所示。<br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<b>算法5.3</b> 基于同策略首次访问$\varepsilon-greedy$策略的MC算法<hr>
&emsp;&emsp;<font size=3.5><b>输入：</b></font>
<font size=3.5>待评估的$\varepsilon-greedy$策略$\pi(a\mid s)$</font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
<font size=3.5>1.&emsp;对所有$s\in\mathcal{S}^+，a\in\mathcal{A}(s)$，初始化$Q(s,a)\in\mathbb{R}$，$Q(s^T,a)=0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;对所有$s\in\mathcal{S}^+，a\in\mathcal{A}(s)$，状态-动作对$(s,a)$被统计的次数$count(s,a)=0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>3.&emsp;$\varepsilon\gets(0,1)$为一个逐步递减的较小的实数</font><br>
<hr>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;<b>repeat</b> 对每个情节$k=0，1，2，\cdots$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>5.&emsp;&emsp;&emsp;根据策略$\pi(a\mid s)$，生成一个情节序列：$S_0,A_0,R_1,...,S_{T-1},A_{T-1},R_T,S_T$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;$G\gets0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;<b>for</b> 本情节中的每一步$t=T-1$ <b>downto</b>&emsp;0&emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;&emsp;&emsp;$G\gets\gamma G+R_{t+1}$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;&emsp;&emsp;&emsp;<b>if</b>&emsp;$S_t,A_t\notin\{S_O,A_0,S_1,A_1,...,S_{T-1},A_{T-1}\}$ <b>then</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>10.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$Q(S_t,A_t)\gets Q(S_t,A_t)+\frac{1}{count(S_t,A_t)}(G-Q(S_t,A_t))$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>11.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$count(S_t,A_t)\gets count(S_t,A_t)+1$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>12.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$A^*\gets arg\max\limits_aQ(S_t,a)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>13.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<b>for</b>  $a\in\mathcal{A}(S_t)$ <b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>14.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<b>if</b>  $a==A^*$ <b>then</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>15.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\pi(a\mid S_t)\gets1-\varepsilon+\frac{\varepsilon}{\mid\mathcal{A}(S_t)\mid}$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>16.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<b>else</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>17.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;$\pi(a\mid S_t)\gets\frac{\varepsilon}{\mid\mathcal{A}(S_t)\mid}$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>18.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<b>end if</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>19.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<b>end for</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>20.&emsp;&emsp;&emsp;&emsp;&emsp;<b>end if</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>21.&emsp;&emsp;&emsp;<b>end for</b></font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输出：</b></font>
&emsp;<font size=3.5>$q_*=Q；\pi_\ast=\pi$</font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>

&emsp;&emsp;在算法5.3中，基于动作值函数$Q$的$\varepsilon-$贪心策略，对于贪心动作给予$\pi(a\mid S_t)\gets1-\varepsilon+\frac{\varepsilon}{\mid\mathcal{A}(S_t)\mid}$的概率，其他动作给予$\pi(a\mid S_t)\gets\frac{\varepsilon}{\mid\mathcal{A}(S_t)\mid}$概率。<br>
&emsp;&emsp;<b>例5.3</b> 利用同策略首次访问$\varepsilon-$贪心策略MC算法，给出扫地机器人的最优策略。扫地机器人通过多次实验，不断的更新$Q$值，最终收敛到最优策略，并得到一条回报最大的路径。同策略蒙特卡洛首次访问控制算法中，对动作值函数$Q$的计算，也是通过对每一情节中第一次访问到的该状态-动作对的回报进行平均，然后选择使该动作值函数$Q$最大的动作，作为该状态下应该采取的动作。基于同策略首次访问MC算法的扫地机器人最优策略计算过程，表5.1给出5个代表性状态的求解过程。<br>

<table align="center" border="3" bgcolor="#DC143C" cellspacing="1" cellpadding="5px" width="100%">
    <caption>
        <font size=2><center>表5.1 基于同策略首次访问MC算法的扫地机器人最优策略计算过程$（\varepsilon_0=0.1）$</center></font>
    </caption>
    <tr>
        <th width="80px"><p style="text-align:center">&emsp;&emsp;</p></th>
        <th><p style="text-align:center">$S_5$</p></td>
        <th><p style="text-align:center">$S_{10}$</p></td>
        <th><p style="text-align:center">$S_{18}$</p></td>
        <th><p style="text-align:center">$S_{20}$</p></td>
        <th><p style="text-align:center">$S_{24}$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_0$</p></td>
        <td><p style="text-align:center">0.00;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;0.00</p></td>
        <td><p style="text-align:center">*.**;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">*.**;0.00;0.00;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_0$</p></td>
        <td><p style="text-align:center">0.933;0.033;0.00;0.033</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.025;0.025;0.925;0.025</p></td>
        <td><p style="text-align:center">0.00;0.950;0.00;0.050</p></td>
        <td><p style="text-align:center">0.00;0.950;0.050;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$A_0$</p></td>
        <td><p style="text-align:center">1;0;0;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;0;1;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_1$</p></td>
        <td><p style="text-align:center">0.00;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">-0.07;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;3.00</p></td>
        <td><p style="text-align:center">*.**;-0.05;*.**;-4.14</p></td>
        <td><p style="text-align:center">*.**;0.00;0.00;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_1$</p></td>
        <td><p style="text-align:center">0.933;0.033;0.00;0.033</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.025;0.025;0.025;0.925</p></td>
        <td><p style="text-align:center">0.00;0.950;0.00;0.050</p></td>
        <td><p style="text-align:center">0.00;0.950;0.050;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$A_1$</p></td>
        <td><p style="text-align:center">1;0;0;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_{7500}$</p></td>
        <td><p style="text-align:center">0.29;1.00;*.**;0.27</p></td>
        <td><p style="text-align:center">0.05;0.63;*.**;0.58</p></td>
        <td><p style="text-align:center">1.56;0.53;0.38;3.00</p></td>
        <td><p style="text-align:center">*.**;0.35;*.**;0.72</p></td>
        <td><p style="text-align:center">*.**;3.00;1.59;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_{7500}$</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.025;0.025;0.025;0.925</p></td>
        <td><p style="text-align:center">0.00;0.050;0.00;0.950</p></td>
        <td><p style="text-align:center">0.00;0.950;0.050;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$A_{7500}$</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_{12500}$</p></td>
        <td><p style="text-align:center">0.31;1.00;*.**;0.30</p></td>
        <td><p style="text-align:center">0.11;0.65;*.**;-0.61</p></td>
        <td><p style="text-align:center">1.60;0.67;0.58;3.00</p></td>
        <td><p style="text-align:center">*.**;0.40;*.**;0.81</p></td>
        <td><p style="text-align:center">*.**;3.00;1.64;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_{12500}$</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.025;0.025;0.025;0.925</p></td>
        <td><p style="text-align:center">0.00;0.050;0.00;0.950</p></td>
        <td><p style="text-align:center">0.00;0.950;0.050;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$A_{12500}$</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_{19999}$</p></td>
        <td><p style="text-align:center">0.31;1.00;*.**;0.30</p></td>
        <td><p style="text-align:center">0.11;0.66;*.**;-0.6</p></td>
        <td><p style="text-align:center">1.61;0.67;0.64;3.00</p></td>
        <td><p style="text-align:center">*.**;0.43;*.**;0.94</p></td>
        <td><p style="text-align:center">*.**;3.00;1.67;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_{19999}$</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.025;0.025;0.025;0.925</p></td>
        <td><p style="text-align:center">0.00;0.050;0.00;0.950</p></td>
        <td><p style="text-align:center">0.00;0.950;0.050;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$A_{19999}$</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_{20000}$</p></td>
        <td><p style="text-align:center">0.31;1.00;*.**;0.30</p></td>
        <td><p style="text-align:center">0.11;0.66;*.**;-0.60</p></td>
        <td><p style="text-align:center">1.61;0.67;0.64;3.00</p></td>
        <td><p style="text-align:center">*.**;0.43;*.**;0.94</p></td>
        <td><p style="text-align:center">*.**;3.00;1.67;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_{20000}$</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.033;0.933;0.00;0.033</p></td>
        <td><p style="text-align:center">0.025;0.025;0.025;0.925</p></td>
        <td><p style="text-align:center">0.00;0.050;0.00;0.950</p></td>
        <td><p style="text-align:center">0.00;0.950;0.050;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$A_{20000}$</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_{*}$</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;0;0;1</p></td>
        <td><p style="text-align:center">0;1;0;0</p></td>
</table>

In [ ]:
from collections import defaultdict
from my_book_gridworld0406 import GridWorldEnv
import numpy as np

class Agent():
    def __init__(self):
        self.Q = {}  # {state:{action:q_value}} 初始化Q ★

    def create_epsilon_greedy_policy(self,nA):
        """
        Creates an epsilon-greedy policy based on Q values.

        Args:
            Q: A dictionary that maps from state -> action values

        Returns:
            A function that takes an observation as input and returns a vector
            of action probabilities.
        """

        def policy_fn(action_list,state,epsilon):
            A = np.zeros(nA, dtype=float)
            valid_nA = len(action_list[state])
            for action in action_list[state]:
                A[action] = epsilon / valid_nA
            best_action = max(self.Q[state], key=self.Q[state].get)  # 获取最大value值对应的key，即得到最大动作值函数对应的动作
            A[best_action] += 1.0-epsilon
            return A

        return policy_fn

    def mc_control_epsilon_greedy(self, env, gamma, max_episode_num):
        # flag = True
        returns_sum = defaultdict(float)
        returns_count = defaultdict(float)
        target_policy = self.create_epsilon_greedy_policy(env.action_space.n)
        num_episode = 0
        for state in range(env.observation_space.n):
            self.initValue(state, env.valid_actions_for_states, randomized=False)
        while num_episode < max_episode_num:
            episode=[]
            state= env.reset()
            while True:
                #env.render()
                probs = target_policy(env.valid_actions_for_states, state,epsilon_by_epsiode(num_episode))
                action = np.random.choice(np.arange(len(probs)), p=probs)
                next_state,reward,done,_=env.step(action)
                episode.append((state,action,reward))
                if done:
                    break
                state=next_state

            num_episode+=1
            # Find all (state, action) pairs we've visited in this episode
            # We convert each state to a tuple so that we can use it as a dict key
            sa_in_episode = set([(x[0], x[1]) for x in episode])
            for state, action in sa_in_episode:
                sa_pair = (state, action)
                # Find the first occurance of the (state, action) pair in the episode
                first_occurence_idx = next(i for i, x in enumerate(episode)
                                            if x[0] == state and x[1] == action)
                # Sum up all rewards since the first occurance
                G = sum([x[2] * (gamma ** i) for i, x in enumerate(episode[first_occurence_idx:])])
                # Calculate average return for this state over all sampled episodes
                returns_sum[sa_pair] += G
                returns_count[sa_pair] += 1.0
                self.__setQValue(state,action,returns_sum[sa_pair] / returns_count[sa_pair])


            if num_episode in print_episodes:
                print("episode:{}".format(num_episode))
                for s in print_states:
                    if s in self.Q.keys():
                        print("{}_Q:".format(s), end="")
                        Q_s = []
                        for a in self.Q[s].keys():
                            Q_s.append(round(self.Q[s][a], 2))
                        print(Q_s)

        return self.Q

    # return a possible action list for a given state
    # def possibleActionsForstate(self, state):
    #  actions = []
    #  # add your code here
    #  return actions

    # if a state exists in Q dictionary
    def __isStateInQ(self, state):
        # 判断空值。有值则返回值，无值则返回None - None is not None = False
        return self.Q.get(state) is not None  # 因为是实例属性，所以要用self.进行引用

    def initValue(self, s, valid_actions_list,randomized=False):  # 初始化Q和E
        # Q[s]为空值时进入判断
        if not self.__isStateInQ(s):
            self.Q[s] = {}  # 初始化Q
            for a in valid_actions_list[s]:  # 遍历所有action_name
                self.Q[s][a] = np.random().random() / 10 if randomized is True else 0.0  # 初始化Q(s,a)；随机一个动作值函数。只有结束状态的Q(s,a) = 0


    """Q的获取与设置方法"""

    def __getQValue(self, s, a):  # ①
        return self.Q[s][a]  # argmax(q)

    def __setQValue(self, s, a, new_q):  # ②
        self.Q[s][a] = new_q

np.random.seed(1)
epsilon_start = 0.5
epsilon_final = 0
epsilon_episodes = 20000
epsilon_by_epsiode = lambda episode_idx: epsilon_start - (epsilon_start - epsilon_final) * min(episode_idx,
                                                                                                   epsilon_episodes) / epsilon_episodes
agent=Agent()
env=GridWorldEnv()
print_states=[5,10,18,20,24]
print_episodes=[1,7500,12500,19999,20000]
Q=agent.mc_control_epsilon_greedy(env=env,gamma=0.8,max_episode_num=20000)

episode:1
5_Q:[0.0, 0.0, 0.0]
10_Q:[-0.07, 0.0, 0.0]
18_Q:[0.0, 0.0, 0.0, 3.0]
20_Q:[-0.05, -4.14]
24_Q:[0.0, 0.0]


&emsp;&emsp;另外，常用$\varepsilon-$柔性策略公式还有以下4种：<br>
&emsp;&emsp;（1）随机贪心策略，基于随机数，用一个较小的阈值$\varepsilon$来控制策略的探索性：<br>
$$\pi(a\mid S_t)\gets\begin{cases}
A^* & 如果random()>\varepsilon\\
random(A) & 如果random()\le\varepsilon
\end{cases} $$
&emsp;&emsp;当随机数大于$\varepsilon$时，选择最大动作值函数对应的动作$A^*$；当随机数小于或等于$\varepsilon$时，随机地选择动作$rand(A)$。<br>
&emsp;&emsp;（2）Boltzmann探索，定义$t$时刻选择动作$A_t$的概率，其公式为：<br>
$$
\pi(A_t\mid S_t)=\frac{e^{Q_t(s_t,A_t)/\tau_t}}{\sum\limits_ae^{Q_t(s_t,A_t)/\tau_t}}
$$
&emsp;&emsp;其中，$\tau_t\ge0$表示温度参数，控制探索的随机性。当$\tau_t\to0$时，选择贪心动作；当$\tau_t\to\infty$时，随机选择动作。<br>
&emsp;&emsp;（3）最大置信上界法。虽然贪心策略选取的动作在当前时刻看起来是最好的，但实际上其他一些没有选取到的动作可能从长远看更好。在这些动作中，通常是根据它们的潜力来选择可能最优的动作，因此在选择动作时，一方面要考虑其估计值最大，另外一方面也要考虑探索长时间没有访问到的动作，以免错过更好的动作。一个有效的方法是按照以下公式来选择动作：<br>
$$
A_t=arg\max\limits_a[Q_t(s,a)+c\sqrt{\frac{\ln t}{N_t(s,a)} }]
$$
&emsp;&emsp;其中，$\ln t$表示$t$的自然对数，$N_t(s,a)$表示当前状态$s$，在时刻$t$之前动作$a$被选择的次数。$c$是一个大于$0$的数，用来控制探索的程度。如果$N_t(s,a)=0$，则动作$a$就被认为是当前状态$s$下满足最大化条件的动作。<br>
&emsp;&emsp;（4）乐观初始值方法。给值函数赋予一个比实际价值大得多些的乐观初始值。这种乐观估计会鼓励不断地选取收益接近估计值的动作。但无论选取哪一种动作，收益都比最初始的估计值小，因此在估计值收敛之前，所有动作都会被多次尝试。即使每一次都按照贪心法选择动作，系统也会进行大量的探索。<br>
### 5.4.4 异策略与重要性采样
&emsp;&emsp;另一种常用的无探索始点蒙特卡洛方法为异策略MC方法。在介绍异策略方法之前，我们先来了解重要性采样，因为几乎所有的异策略方法都使用到重要性采样。所谓的重要性采样是利用来自其他分布的样本，估计当前某种分布期望值的通用方法。在引出异策略MC控制之前，首先阐述预测问题的重要性采样（importance sampling）。<br>
&emsp;&emsp;（1）重要性采样<br>
&emsp;&emsp;以离散型数据为例，假设$f(x)$是一个服从$p(x)$分布的函数，其期望公式为：<br>
$$
\mathbb{E}_{x\sim p}[f(x)]=\sum_xp(x)f(x)
$$
其中，$x\sim p$表示$x$服从$p(x)$分布，也可记为$f(x)\sim p$。<br>
&emsp;&emsp;通常情况下，可以在服从$p(x)$分布的离散型数据中进行采样，得到样本集$\{x_1,x_2,...,x_N\}$，则$f(x)$在$p(x)$分布下的期望为：<br>
$$
\mathbb{E}_{x\sim p}[f(x)]\approx\frac{1}{N}\sum_{x_i\sim p,i=1}^Nf(x_i)
$$
&emsp;&emsp;在有些任务中，为了得到分布函数$p(x)$，需要采集大量的样本才能拟合原期望，或存在部分极端、无法代表分布的样本。针对这些任务，在服从$p(x)$分布的数据中采样，存在困难的问题，根据重要性采样原则，可以将该任务转化为从服从简单分布$q(x)$的数据中进行采样，得到的样本集为$\{x_1,x_2,...,x_N\}$。此时$f(x)$在$p(x)$分布下的期望为：<br>
$$
\mathbb{E}_{x\sim p}[f(x)]=\sum_xp(x)f(x)=\sum_xq(x)[\frac{p(x)}{q(x)}f(x)]=\mathbb{E}_{x\sim q}[\frac{p(x)}{q(x)}f(x)] \tag{5.6}
$$
其中，$\mathbb{E}_{x\sim q}[\frac{p(x)}{q(x)}f(x)]$表示函数$\frac{p(x)}{q(x)}f(x)$在$q(x)$分布下的期望。<br>
&emsp;&emsp;根据MC采样思想，在采样数据足够多时， $f(x)$在$p(x)$分布下的期望近似为：<br>
$$
\mathbb{E}_{x\sim p}[f(x)]=\mathbb{E}_{x\sim p}[\frac{p(x)}{q(x)}f(x)]\approx\frac{1}{N}\sum\limits_{x_i\sim q,i=1}^{N}\frac{p(x_i)}{q(x_i)}f(x_i)=\frac{1}{N}\sum\limits_{x_i\sim q,i=1}^{N}\omega(x_i)f(x_i) \tag{5.7}
$$
这里$\omega(x)$为重要性采样比率（importance-sampling ratio），有$\omega(x)=p(x)/q(x)$。<br>
&emsp;&emsp;由此，我们将求解$f(x)$在$p(x)$分布下的函数期望问题，转换为求解包含重要性采样比率$\omega$的$f(x)$在$q(x)$分布下的函数期望。<br>
&emsp;&emsp;重要性采样的特点：<br>
&emsp;&emsp;$\blacktriangleright$&emsp;$q(x)$与$p(x)$具有相同的定义域。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;采样概率分布$q(x)$与原概率分布$p(x)$越接近，方差越小；反之，方差越大。<br>
&emsp;&emsp;通常采用加权重要性采样来减小方差，即用$\sum\limits_{j=1}^{N}\omega(x_j)$替换$N:$<br>
&emsp;&emsp;$\blacktriangleright$&emsp;普通重要性采样（ordinary importance sampling）的函数估计为：<br>
$$
\mathbb{E}_{x\sim p}[f(x)]\approx\sum\limits_{x_i\sim q,i=1}^{N}\omega(x_i)f(x_i)/N  \tag{5.8}
$$
&emsp;&emsp;$\blacktriangleright$&emsp;加权重要性采样（weighted importance sampling）的函数估计为：<br>
$$
\mathbb{E}_{x\sim p}[f(x)]\approx\sum\limits_{x_i\sim q,i=1}^{N}\omega(x_i)f(x_i)/\sum\limits_{j=1}^{N}\omega(x_j) \tag{5.9}
$$
&emsp;&emsp;（2）基于重要性采样的异策略方法<br>
&emsp;&emsp;异策略方法目标策略和行为策略是不同的。这里假设目标策略为$\pi$，行为策略为$b$，所有情节都遵循行为策略$b$，并利用行为策略$b$产生的情节来评估目标策略。这样需要满足覆盖条件，即目标策略$\pi$中的所有动作都会在行为策略$b$中被执行。也就是说，所有满足$\pi(a\mid s)>0$的$(s,a)$均有$b(a\mid s)>0$。根据轨迹在两种策略下产生的相对概率来计算目标策略$\pi$的回报值，该相对概率称为重要性采样比率，记为$\rho$。<br>
&emsp;&emsp;以$S_t$作为初始状态，其采样得到的后续状态-动作对序列为：$A_t,S_{t+1},...,S_{T-1},S_T$。在任意目标策略$\pi$下发生的概率如式（5.10）所示，在任意行为策略$b$下发生后的概率如式（5.11）所示：<br>
$$\begin{aligned}
P(A_t,S_{t+1},A_{t+1},...,S_T\mid S_t,A_{t:T-1}\sim\pi)&=\pi(A_t\mid S_t)p(S_{t+1}\mid S_{t+1},A_t)\pi(A_{t+1}\mid S_{t+1})...p(S_T\mid S_{T-1,A_{T-1}})\\
&=\prod_{k=t}^{T-1}\pi(A_k\mid S_k)p(S_{k+1}\mid S_k,A_k)
\end{aligned} \tag{5.10} $$
$$\begin{aligned}
P(A_t,S_{t+1},A_{t+1},...,S_T\mid S_t,A_{t:T-1}\sim b)&=b(A_t\mid S_t)p(S_{t+1}\mid S_{t+1},A_t)b(A_{t+1}\mid S_{t+1})...p(S_T\mid S_{T-1,A_{T-1}})\\
&=\prod_{k=t}^{T-1}b(A_k\mid S_k)p(S_{k+1}\mid S_k,A_k)
\end{aligned} \tag{5.11} $$
其中，$p$表示状态转移概率，$T$是该情节的终止时刻。注意：公式累乘符号的上标为$T-1$，因为最后一个动作发生在$T-1$时刻。$S_t,A_{t:T-1}\sim\pi$表示该情节服从目标策略$\pi$。<br>
&emsp;&emsp;这样某一情节在目标策略和行为策略下发生的相对概率为：<br>
$$
\rho_{t:T-1}=\frac{\prod\limits_{k=t}^{T-1}\pi(A_k\mid S_k)p(S_{k+1}\mid S_k,A_k)}{\prod\limits_{k=t}^{T-1}b(A_k\mid S_k)p(S_{k+1}\mid S_k,A_k)}=\prod_{k=t}^{T-1}\frac{\pi(A_k\mid S_k)}{b(A_k\mid S_k)}
\tag{5.12}$$
其中，$\rho_{t:T-1}$表示某一情节从$t$到$T$时刻的重要性采样比率，也就是基于两种策略采取动作序列$A_t,A_{t+1},...,A_{T-1}$的相对概率，与重要性采样比率$\omega(x)$相对应。从式（5.12）可以看出，尽管情节的形成依赖于状态转移概率$p$，但由于分子分母中同时存在$p$，可以被消去，所以重要性采样比率仅仅依赖于两个策略，而与状态转移概率无关。<br>
&emsp;&emsp;行为策略中的回报期望是不能直接用于评估目标策略的。根据重要性采样原则，需要使用比例系数$\rho_{t:T-1}$对回报进行调整，使其矫正为正确的期望值：<br>
$$
\mathbb{E}[G_t\mid S_t=s]=v_b(s)\Rightarrow v_\pi(s)=\mathbb{E}[\rho_{t:T-1}G_t\mid S_t=s] \tag{5.13}
$$
&emsp;&emsp;假设遵循行为策略$b$采样得到了一系列情节。为方便计算，将这些情节首位相连，并按时刻状态出现的顺序进行编号。例如第1个情节在时刻100状态结束，则第2个情节的编号就在时刻101状态开始，以此类推。在每次访问方法中，存储所有访问过状态$s$的时间步，记为$\mathcal{T}(s)$，并以$\mid\mathcal{T}(s)\mid$表示状态$s$被访问过的总次数。在首次访问方法中，$\mathcal{T}(s)$只包括这些情节中第一次访问到$s$的时间步。此外，以$T(t)$表示在$t$时刻后的第一个终止时刻，以$G_t$表示从$t$到$T(t)$时刻的回报，以$\rho_{t:T-1}$表示回报$G_t$的重要性采样比率（在增量式计算中常简写为$W_t$）。<br>
&emsp;&emsp;根据重要性采样思想，状态值函数的计算方法分为两种：<br>
&emsp;&emsp;$\blacktriangleright$&emsp;普通重要性采样（ordinary importance sampling）<br>
&emsp;&emsp;将回报按照权重缩放后进行平均。属于无偏估计，具有方差无界的特点。其计算如式（5.14）所示：<br>
$$
V(s)=\frac{\sum\limits_{t\in\mathcal{T}(s)}\rho_{t:T(t)-1}G_t}{\mid\mathcal{T}(s)\mid} \tag{5.14}
$$
&emsp;&emsp;$\blacktriangleright$&emsp;加权重要性采样（weighted importance sampling）<br>
&emsp;&emsp;将回报进行加权平均。属于有偏估计，具有方差较小的特点。其计算如式（5.15）所示：<br>
$$
V(s)=\frac{\sum\limits_{t\in\mathcal{T}(s)}\rho_{t:T(t)-1}G_t}{\sum\limits_{t\in\mathcal{T}(s)}\rho_{t:T(t)-1}} \tag{5.15}
$$
分母为$0$时，$V(s)=0$。<br>
&emsp;&emsp;两种方法的主要差异在于偏差和方差的不同。后面将讨论它们在首次访问方法下，获得单一情节回报后的值函数估计值。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;普通重要性采样的偏差与方差<br>
&emsp;&emsp;采用某种方法估计值函数，当估计结果的期望恒为$v_\pi(s)$时，该方法是无偏估计，但其方差可能是无界的。当$\rho=10$时，表明该轨迹在目标策略下发生的可能性是行为策略下的$10$倍，$V(s)=10G_t$，根据普通重要性采样得到的估计值$V_\pi(x)$是回报值的$10$倍，这就存在了高方差。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;加权重要性采样的偏差与方差<br>
&emsp;&emsp;由于比例系数$\rho$被消去，所以加权重要性采样的估计值就等于回报值，与重要性采样比例无关。因为该回报值是仅有的观测结果，所以是一个合理的估计，但它的期望是$v_b(s)$而非$v_\pi(s)$，所以该方法属于有偏估计。此外，由于加权估计中回报的最大权重是1，所以其方差会明显低于普通估计。<br>
&emsp;&emsp;而实际上，由于重要性采样比率涉及到所有状态的转移概率，因此有很高的方差，从这一点来说，MC算法不太适合于处理异策略问题。异策略MC只有理论研究价值，实际应用的效果并不明显，难以获得最优动作值函数。<br>

### 5.4.5 蒙特卡洛中的增量式计算
&emsp;&emsp;增量式计算在强化学习中具有很重要的应用价值。本节在说明MC增量式计算之前，先来了解经典的增量式计算。<br>
&emsp;&emsp;（1）经典的增量式计算<br>
&emsp;&emsp;计算一组数据的均值，最简单的方法是使用采样平均法，即将所有的历史信息记录下来，然后求平均。假设有一组实数数据，其形式为：$x_1,x_2,...,x_k,x_{k+1},...。$。令$x_k$为第$k$个数据的数值，$u_k$为前$k$个数据的平均值，即有：<br>
$$
u_k=\frac{1}{k}\sum_{i=1}^kx_i  \tag{5.16}
$$
&emsp;&emsp;使用式（5.16）计算前$k$个数据的平均值时，需要对这$k$个数据进行存储，然后再做平均。这种方法容易产生计算和存储消耗量大的问题。根据数学中的迭代思想，引入增量式计算方法，以简化求解过程。增量式推导如下所示：<br>
$$\begin{aligned}
u_k&=\frac{1}{k}\sum_{i=1}^kx_i\\
&=\frac{1}{k}(x_k+\sum_{i=1}^{k-1}x_i)\\
&=\frac{1}{k}(x_k+(k-1)u_{k-1})\\
&=u_{k-1}+\frac{1}{k}(x_k-u_{k-1})\\
\end{aligned}  $$
&emsp;&emsp;根据上式的规律，构建经典增量式公式，如式（5.17）所示：<br>
$$
NewEstimate\gets OldEstimate+StepSize(Target-OldEstimate)  \tag{5.17}
$$
&emsp;&emsp;其中对式（5.17）中变量说明如下<br>
&emsp;&emsp;$\blacktriangleright$&emsp;$Target:$表示当前采样值，称其为目标值，通常伴随着噪声，即存在偏差；<br>
&emsp;&emsp;$\blacktriangleright$&emsp;$OldEstimate，NewEstimate:$表示真实值的估计值，可理解为$V$或$Q$，其目的是逼近真实目标值$v$或$q$；<br>
&emsp;&emsp;$\blacktriangleright$&emsp;$Target-OldEstimate:$表示评估误差，通常记为$\delta$。在递增公式中，通过向$Target$趋近而使得$\delta$逐步减少；<br>
&emsp;&emsp;$\blacktriangleright$&emsp;$StepSize:$表示步长（学习率），通常记为$\alpha$。在这里用一个较小值$\alpha$来替代$1/k$，使公式更具一般性。<br>
&emsp;&emsp;需要说明的是，之所以将$Target$称为目标值，是因为从单次更新过程来看，$OldEstimate$是朝着$Target$移动的。而从整个更新结果来看，$OldEstimate$是朝着真实目标值移动的。在自举方法中，$Target$也常被称为自举估计值$（bootstrapping estimate）$<br>
&emsp;&emsp;由此可见，增量式计算方法是一种基于样本$Target$的随机近似过程，拆分了均值求解过程，减少了存储消耗，简化了计算过程。<br>
&emsp;&emsp;（2）MC的增量式<br>
&emsp;&emsp;对于MC预测问题，在采集情节的基础上也可以使用增量式的实现方法。其增量式的实现可以分为同策略和异策略两种模式。<br>
&emsp;&emsp;同策略MC：使用传统增量式计算公式，不涉及重要性采样，$t$时刻状态$S_t$的状态值函数更新递归式如下所示：<br>
$$
V(S_t)\gets V(S_t)+\alpha(G_t-V(S_t)) \tag{5.18}
$$
&emsp;&emsp;$\blacktriangleright$&emsp;公式右侧的$V(S_t)$为历史状态值函数的均值，表示估计值，即$OldEstimate;$<br>
&emsp;&emsp;$\blacktriangleright$&emsp;$G_t$为$t$时刻的回报，表示目标值，即$Target;$<br>
&emsp;&emsp;$\blacktriangleright$&emsp;$\alpha$为步长，当$\alpha$是固定步长时，该式称为恒定$-\alpha  MC$。<br>
&emsp;&emsp;异策略MC：假设已经获得了状态$s$的回报序列$G_1,G_2,...,G_{n-1}$，每个回报都对应一个随机重要性权重$W_i(W_i=\rho_{t:T(t)-1})$。当获得新的回报值$G_n$时，我们希望以增量式的方式，在状态值函数估计值$V_n$的基础上估计$V_{n+1}$。<br>
&emsp;&emsp;在普通重要性采样中，仅仅需要对回报赋予权重$W_i$，其增量式与经典增量式方程基本一致：<br>
$$
V(s)\gets V(s)+\alpha(WG-V(s)) \tag{5.19}
$$
&emsp;&emsp;在加权重要性采样中，需要为每一个状态计算前$n$个回报的累积权重$C_n$：<br>
$$
C_n=\sum_{k=1}^nW_k\Rightarrow C_n=C_{n-1}+W_n\quad\quad(C_0=0)\tag{5.20}
$$
其中，$C_n$表示前$n$个回报的重要性权重之和，右式是其增量形式。由此得到$V_n$的更新递归式为：<br>
$$
V_{n+1}=V_n+\frac{W_n}{C_n}(G_n-V_n)\quad\quad(n\ge 1)\tag{5.21}
$$
推导过程为：
$$\begin{aligned}
V_{n+1}&=\sum_{k=1}^nW_kG_k/C_n=(\sum_{k=1}^{n-1}W_kG_k+W_nG_n)/C_n\\
&=\frac{\sum\limits_{k=1}^{n-1}W_kG_k}{C_n}+\frac{W_nG_n}{C_n}=\frac{\sum\limits_{k=1}^{n-1}W_kG_k}{\sum\limits_{k=1}^{n-1}W_k}*\frac{\sum\limits_{k=1}^{n-1}W_k}{C_n}+\frac{W_nG_n}{C_n}\\
&=V_n\frac{C_n-W_n}{C_n}+\frac{W_nG_n}{C_n}\\
&=V_n+\frac{W_n}{C_n}(G_n-V_n)\quad\quad(n\ge 1)\\
\end{aligned}  $$
&emsp;&emsp;（3）随机近似条件<br>
&emsp;&emsp;由于MC是基于采样方法进行更新的，当状态和动作空间是离散且有穷时，只要满足如下两个条件，动作值函数估计值$Q_\pi(s,a)$就能收敛于真实动作值函数$q_\pi(s,a)$。<br>
&emsp;&emsp;$\blacktriangleright$&emsp;$$\sum\limits_{t=1}^\infty\alpha_t=\infty,\sum\limits_{t=1}^\infty\alpha_t^2<\infty \tag{5.22}$$
&emsp;&emsp;$\blacktriangleright$&emsp;所有的状态-动作对都能够渐近地被无限次访问到。<br>
&emsp;&emsp;若$\alpha$根据随机近似条件逐渐变小，则它能以概率$1$收敛。以上收敛条件均适用于其他采样模型。所以只要所有$(s,a)$都能被无限多次地访问到，且贪心策略在极限情况下能够收敛（$\varepsilon$衰减为0），那么采样模型就能以$1$的概率收敛到最优动作值函数和最优策略。<br>

### 5.4.6 异策略蒙特卡洛控制
&emsp;&emsp;异策略MC控制算法与异策略MC预测算法的原理一致，动作值函数更新递归式为：<br>
$$
Q(S_t,A_t)\gets Q(S_t,A_t)+\frac{W}{C(S_t,A_t)}[G-Q(S_t,A_t)]  \tag{5.23}
$$

&emsp;&emsp;算法5.4给出了用于估算最优策略的，异策略每次访问MC控制算法。<br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" />
&emsp;&emsp;<font size=3.5><b>算法5.4</b> 异策略每次访问MC控制算法</font>
<br>
<hr>
&emsp;&emsp;<font size=3.5><b>初始化：</b></font>
<font size=3.5>1.&emsp;对所有$s\in\mathcal{S}^+，a\in\mathcal{A}(s)$，初始化$Q（s,a）\in\mathbb{R}$，$Q(s^T,a)=0,C(s,a)=0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>2.&emsp;$\varepsilon\gets(0,1)$为一个逐步递减的较小的实数</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>3.&emsp;$\pi(s)\gets arg\max\limits_aQ(s,a)$</font><br>
<hr>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>4.&emsp;<b>repeat</b> 对每个情节$k=0，1，2，\cdots$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>5.&emsp;&emsp;&emsp;$b\gets$任意软性策略</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>6.&emsp;&emsp;&emsp;根据策略$b(s)$，从初始状态-动作对$(S_0,A_0)$开始，生成一个情节序列：$S_0,A_0,R_1,...,S_{T-1},A_{T-1},R_T,S_T$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>7.&emsp;&emsp;&emsp;$G\gets0$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>8.&emsp;&emsp;&emsp;$W\gets1$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>9.&emsp;&emsp;&emsp;<b>for</b> 本情节中的每一步$t=T-1$ <b>downto</b>&emsp;0&emsp;<b>do</b></font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>10.&emsp;&emsp;&emsp;&emsp;$G\gets\gamma G+R_{t+1}$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>11.&emsp;&emsp;&emsp;&emsp;$C(S_t,A_t)\gets C(S_t,A_t)+W$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>12.&emsp;&emsp;&emsp;&emsp;$Q(S_t,A_t)\gets Q(S_t,A_t)+\frac{w}{C(S_t,A_t)}[G-Q(S_t,A_t)]$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>13.&emsp;&emsp;&emsp;&emsp;$\pi(S_t)\gets arg\max\limits_aQ(S_t,a)$</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>14.&emsp;&emsp;&emsp;&emsp;<b>if</b> $A_t=\pi(S_t)$ <b>then</b> $W\gets W\frac{1}{b(A_t\mid S_t)}$<br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<b>else</b> break</font><br>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<font size=3.5>15.&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;<b>end for</b></font><br>
<hr>
&emsp;&emsp;<font size=3.5><b>输出：</b></font>
&emsp;<font size=3.5>$\pi_\ast=\pi$<br></font><br>
<hr style="height:1px;border:none;border-top:1px solid #555555;" /><br>

&emsp;&emsp;在算法5.4第14行，如果行为策略$b$采样的动作与目标策略$\pi$采取的动作不同，则$\pi(A_t\mid S_t)=0$，重要性权重为0，无法更新值函数，退出本次情节的迭代。从这一步可以看出，每一次迭代都会很快停止，效率不会很高，这一点也是异策略MC应用性不强的原因之一；第15行：当$A_t=\pi(S_t)$时，$\pi(A_t\mid S_t)=1$，重要性权重更新递归式就简写为了$W\gets W\frac{1}{b(A_t\mid S_t)}$。<br>
&emsp;&emsp;<b>例5.4</b> 对于扫地机器人问题，通过行为策略来生成情节，然后利用每次访问和重要性采样比率计算动作值函数$Q(S_t,A_t)$，如果行为策略采样的动作不是目标策略采取的动作，则会结束该循环开始新一轮循环。这样也就产生很多无用的数据，使得学习效率不高。表5.2为异策略每次访问MC控制算法的$Q$值更新表。<br>

<table align="center" border="3" bgcolor="#DC143C" cellspacing="1" cellpadding="5px" width="100%">
    <caption>
        <font size=2><center>表5.2 异策略每次访问MC控制算法的$q$值更新过程</center></font>
    </caption>
    <tr>
        <th width="80px"><p style="text-align:center">&emsp;&emsp;</p></th>
        <th><p style="text-align:center">$S_5$</p></td>
        <th><p style="text-align:center">$S_{10}$</p></td>
        <th><p style="text-align:center">$S_{18}$</p></td>
        <th><p style="text-align:center">$S_{20}$</p></td>
        <th><p style="text-align:center">$S_{24}$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_0$</p></td>
        <td><p style="text-align:center">0.00;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;0.00</p></td>
        <td><p style="text-align:center">*.**;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">*.**;0.00;0.00;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_0$</p></td>
        <td><p style="text-align:center">0.17;0.17;0.00;0.67</p></td>
        <td><p style="text-align:center">0.17;0.17;0.00;0.67</p></td>
        <td><p style="text-align:center">0.12;0.12;0.62;0.12</p></td>
        <td><p style="text-align:center">0.00;0.25;0.00;0.75</p></td>
        <td><p style="text-align:center">0.00;0.25;0.75;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_1$</p></td>
        <td><p style="text-align:center">0.00;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">0.00;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">1.92;0.00;0.00;0.00</p></td>
        <td><p style="text-align:center">*.**;0.00;*.**;0.00</p></td>
        <td><p style="text-align:center">*.**;3.00;0.00;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_1$</p></td>
        <td><p style="text-align:center">0.17;0.17;0.00;0.67</p></td>
        <td><p style="text-align:center">0.17;0.17;0.00;0.67</p></td>
        <td><p style="text-align:center">0.63;0.12;0.12;0.12</p></td>
        <td><p style="text-align:center">0.00;0.25;0.00;0.75</p></td>
        <td><p style="text-align:center">0.00;0.75;0.25;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_{7500}$</p></td>
        <td><p style="text-align:center">0.96;1.00;*.**;0.98</p></td>
        <td><p style="text-align:center">1.22;0.80;*.**;1.21</p></td>
        <td><p style="text-align:center">1.92;1.89;1.89;3.00</p></td>
        <td><p style="text-align:center">*.**;1.18;*.**;1.23</p></td>
        <td><p style="text-align:center">*.**;3.00;1.92;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_{7500}$</p></td>
        <td><p style="text-align:center">0.10;0.79;0.00;0.10</p></td>
        <td><p style="text-align:center">0.79;0.10;0.00;0.10</p></td>
        <td><p style="text-align:center">0.08;0.08;0.08;0.77</p></td>
        <td><p style="text-align:center">0.00;0.16;0.00;0.84</p></td>
        <td><p style="text-align:center">0.00;0.84;0.16;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_{12500}$</p></td>
        <td><p style="text-align:center">0.98;1.00;*.**;0.98</p></td>
        <td><p style="text-align:center">1.22;0.80;*.**;1.22</p></td>
        <td><p style="text-align:center">1.92;1.90;1.90;3.00</p></td>
        <td><p style="text-align:center">*.**;1.2;*.**;1.23</p></td>
        <td><p style="text-align:center">*.**;3.00;1.91;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_{12500}$</p></td>
        <td><p style="text-align:center">0.06;0.88;0.00;0.06</p></td>
        <td><p style="text-align:center">0.12;0.44;0.00;0.44</p></td>
        <td><p style="text-align:center">0.05;0.05;0.05;0.86</p></td>
        <td><p style="text-align:center">0.00;0.09;0.00;0.91</p></td>
        <td><p style="text-align:center">0.00;0.91;0.09;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
        <td><p style="text-align:center">$\vdots$</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_{19999}$</p></td>
        <td><p style="text-align:center">0.98;1.00;*.**;0.98</p></td>
        <td><p style="text-align:center">1.22;0.80;*.**;1.22</p></td>
        <td><p style="text-align:center">1.92;1.91;1.91;3.00</p></td>
        <td><p style="text-align:center">*.**;1.21;*.**;1.23</p></td>
        <td><p style="text-align:center">*.**;3.00;1.92;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_{19999}$</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
        <td><p style="text-align:center">0.50;0.00;0.00;0.50</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$Q_{20000}$</p></td>
        <td><p style="text-align:center">0.50;1.00;*.**;0.45</p></td>
        <td><p style="text-align:center">0.62;0.72;*.**;0.58</p></td>
        <td><p style="text-align:center">1.78;1.61;1.69;3.00</p></td>
        <td><p style="text-align:center">*.**;1.02;*.**;1.21</p></td>
        <td><p style="text-align:center">*.**;3.00;1.92;*.**</p></td>
    </tr>
    <tr>
        <td><p style="text-align:center">$\pi_{20000}$</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
        <td><p style="text-align:center">0.50;0.00;0.00;0.50</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
    </tr>
     <tr>
        <td><p style="text-align:center">$\pi_{*}$</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
        <td><p style="text-align:center">0.50;0.00;0.00;0.50</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;0.00;0.00;1.00</p></td>
        <td><p style="text-align:center">0.00;1.00;0.00;0.00</p></td>
</table>

In [ ]:
from collections import defaultdict
from my_book_gridworld0406 import GridWorldEnv
import numpy as np

class Agent():
    def __init__(self):
        self.Q = {}  # {state:{action:q_value}} 初始化Q ★
        self.C = {}

    def create_random_policy(self,nA):
        """
        Creates a random policy function.

        Args:
            nA: Number of actions in the environment.

        Returns:
            A function that takes an observation as input and returns a vector
            of action probabilities
        """

        def policy_fn(action_list,observation):
            A = np.zeros(nA, dtype=float)
            valid_nA=len(action_list[observation])
            for action in action_list[observation]:
                A[action]=1./valid_nA
            return A

        return policy_fn

    def create_greedy_policy(self,nA):
        """
        Creates a greedy policy based on Q values.

        Args:
            Q: A dictionary that maps from state -> action values

        Returns:
            A function that takes an observation as input and returns a vector
            of action probabilities.
        """

        def policy_fn(state):
            A = np.zeros(nA, dtype=float)
            best_action = max(self.Q[state], key=self.Q[state].get)  # 获取最大value值对应的key，即得到最大动作值函数对应的动作
            A[best_action] = 1.0
            return A

        return policy_fn

    def mc_control_importance_sampling(self, env, gamma, max_episode_num):
        behavior_policy=self.create_random_policy(env.action_space.n)
        target_policy=self.create_greedy_policy(env.action_space.n)
        num_episode = 0
        while num_episode <= max_episode_num:
            episode=[]
            state= env.reset()

            for t in range(100):
                if not self.__isStateInQ(state):
                    self.initValue(state, env.valid_actions_for_states, randomized=False)
                if not self.__isStateInC(state):
                    self.initCValue(state, env.valid_actions_for_states)
                probs=behavior_policy(env.valid_actions_for_states,state)
                action = np.random.choice(np.arange(len(probs)), p=probs)
                next_state,reward,done,_=env.step(action)
                episode.append((state,action,reward))
                if done:
                    # print(t)
                    break
                state=next_state

            num_episode+=1
            # Sum of discounted returns
            G = 0.0
            # The importance sampling ratio (the weights of the returns)
            W = 1.0
            # For each step in the episode, backwards
            for t in range(len(episode))[::-1]:
                state, action, reward = episode[t]
                # Update the total reward since step t
                G = gamma * G + reward
                # Update weighted importance sampling formula denominator
                pre_c=self.__getCValue(state,action)
                self.__setCValue(state,action,pre_c+W)
                # Update the action-value function using the incremental update formula (5.7)
                # This also improves our target policy which holds a reference to Q
                now_c=self.__getCValue(state,action)
                pre_Q=self.__getQValue(state,action)
                self.__setQValue(state,action,pre_Q+(W / now_c) * (G - pre_Q))
                # If the action taken by the behavior policy is not the action
                # taken by the target policy the probability will be 0 and we can break
                if action != np.argmax(target_policy(state)):
                    break
                W = W * 1. / behavior_policy(env.valid_actions_for_states,state)[action]

            if num_episode in print_episodes:
                print("episode:{}".format(num_episode))
                for s in print_states:
                    if s in self.Q.keys():
                        print("{}_Q:".format(s), end="")
                        Q_s = []
                        for a in self.Q[s].keys():
                            Q_s.append(round(self.Q[s][a], 2))
                        print(Q_s)
        return self.Q

    # return a possible action list for a given state
    # def possibleActionsForstate(self, state):
    #  actions = []
    #  # add your code here
    #  return actions

    # if a state exists in Q dictionary
    def __isStateInQ(self, state):
        # 判断空值。有值则返回值，无值则返回None - None is not None = False
        return self.Q.get(state) is not None  # 因为是实例属性，所以要用self.进行引用

    # if a state exists in C dictionary
    def __isStateInC(self, state):
        # 判断空值。有值则返回值，无值则返回None - None is not None = False
        return self.C.get(state) is not None  # 因为是实例属性，所以要用self.进行引用

    def initValue(self, s, valid_actions_list,randomized=False):  # 初始化Q
        # Q[s]为空值时进入判断
        if not self.__isStateInQ(s):
            self.Q[s] = {}  # 初始化Q
            for a in valid_actions_list[s]:  # 遍历所有action_name
                self.Q[s][a] = np.random.random() / 10 if randomized is True else 0.0  # 初始化Q(s,a)；随机一个动作值函数。只有结束状态的Q(s,a) = 0

    def initCValue(self, s, valid_actions_list):  # 初始化C
        if not self.__isStateInC(s):
            self.C[s] = {}  # 初始化C
            for a in valid_actions_list[s]:  # 遍历所有action_name
                self.C[s][a] = 0.0  # 初始化C(s,a)值为0.0

    """Q的获取与设置方法"""

    def __getQValue(self, s, a):  # ①
        return self.Q[s][a]  # argmax(q)

    def __setQValue(self, s, a, new_q):  # ②
        self.Q[s][a] = new_q

    """C的获取与设置方法"""

    def __getCValue(self, s, a):
        return self.C[s][a]

    def __setCValue(self, s, a, new_c):
        self.C[s][a] = new_c

np.random.seed(1)
agent=Agent()
env=GridWorldEnv()
print_states=[5,10,18,20,24]
print_episodes=[1,7500,12500,19999,20000]
Q=agent.mc_control_importance_sampling(env=env,gamma=0.8,max_episode_num=20000)

## 5.4 本章小结
&emsp;&emsp;本章介绍了从经验中学习价值函数和最优策略的蒙特卡洛方法，这些“经验”主要体现在从多个情节采样数据。与DP方法相比，其优势主要在以下3个方面：（1）MC方法不需要完整的环境动态模型，而可以直接通过与环境的交互来学习最优的决策行为。（2）MC方法可以使用数据仿真或采样模型。在很多应用中，构建DP方法所需要的显式状态概率转移模型通常很困难，但是通过仿真采样得到多情节序列数据却很简单。（3）MC方法可以简单、高效地聚焦于状态的一个小的子集，它可以只评估关注的区域而不评估其他的状态。